In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math; import os
from torch.distributions import Normal
from torch.optim import Adam
from gym import spaces 
import numpy as np 
from IPython.display import clear_output
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
import gym
from gym import spaces; import random
import numpy as np; import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.multioutput import MultiOutputRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn import datasets
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import _tree
import matplotlib.pyplot as plt
from scipy.stats import norm

hidden_size = [1000,1600,1000]
lr = 0.0001
batch_size = 144*1
tau = 0.005
LOG_SIG_MAX = 2
LOG_SIG_MIN = -20
epsilon = 1e-6

args= {}
args['alpha'] = 0.2
args['gamma'] = 0.9995
args['tau'] = 0.005
args['lr'] = 0.00003
args['batch_size'] = 144*10
args['num_steps'] = 1000001
args['hidden_size'] = [1000,1600,1000]
args['updates_per_step'] = 5
args['start_steps'] = 10000
args['replay_size'] = 144*2000
args['policy'] = 'Gaussian'
args['automatic_entropy_tuning'] = False
args['target_update_interval'] = 5
args['cuda'] = 'store_true'

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
def cool_stp(Temp_mean_p0,Temp_mean_m1,sen_hou,rh,rtp_2):
    if Temp_mean_p0 <= 24.427:
        if rtp_2 <= 0.15:
            if rh <= 55.0:
                if Temp_mean_p0 <= 20.583:
                    return 23.953
                else:
                    return 25.243
            else:
                if sen_hou <= 6.0:
                    return 27.798
                else:
                    return 25.333
        else:
            if Temp_mean_p0 <= 23.315:
                if Temp_mean_m1 <= 26.08:
                    return 24.535
                else:
                    return 23.603
            else:
                if Temp_mean_m1 <= 26.054:
                    return 23.351
                else:
                    return 21.897
    else:
        if sen_hou <= 20.4:
            if Temp_mean_p0 <= 25.561:
                if sen_hou <= 18.0:
                    return 21.799
                else:
                    return 24.005
            else:
                if sen_hou <= 8.4:
                    return 20.518
                else:
                    return 21.073
        else:
            if Temp_mean_m1 <= 22.687:
                if Temp_mean_m1 <= 20.928:
                    return 22.374
                else:
                    return 23.759
            else:
                if Temp_mean_p0 <= 27.436:
                    return 27.003
                else:
                    return 23.155

In [3]:
states = np.load('states_n.npy')
actions = np.load('actions_n.npy')

In [7]:
len(states)

300000

In [16]:
agent2_feature_names = ['Temp_mean_p0','Temp_mean_m1','sen_hou','temp_OA_0','temp_OA_1','rh','rtp_0','rtp_1','rtp_2','rtp_3','rtp_4','rtp_5','rtp_6']

def batt_state_rbc(hours,rtp_0,rtp_1,rtp_2):
    if rtp_2>0.2 or rtp_1>0.2: 
        batt_act = 1.0       
    else:
        batt_act = 0           
    if hours>15:
        batt_act = 0.5 

    if rtp_0 >0.25:
        batt_act = -1  

    if hours>14:
        batt_act = 1 

    return batt_act 


building_id=1

min_max = {}

min_max['sen_hou'] = [0,24]
min_max['temp_OA'] = [-20,40]
min_max['rh_OA'] = [0,100]
min_max['Temp_mean_p0'] = [15,30]; min_max['Temp_mean_m1'] = [15,30]
min_max['sen_hou'] = [0,24]
min_max['hir_sol'] = [0,700] 

if building_id == 1: 
    data = '0_Data/Case_1/comb_data.csv'
    min_max["hvac_dem_tot"] = [0,6020] #1  #kW 
    min_max['appliances_pow'] = [0,42] #1 #kW
    min_max['elec_lights'] = [0,70]   #1 #kW
    min_max['Occ'] = [0,3300]
    min_max['batt_cap'] = [0,6000] 
    min_max['rtp'] = [0.05,0.3]
    min_max['soc'] = [0,1]
    scale_delta, scale_hvac = 200 , 200
elif building_id == 6:
    data = '0_Data/Case_6/comb_data.csv'
    min_max["hvac_dem_tot"] = [0,1620] #6  #kW 
    min_max['appliances_pow'] = [0,160] #6 #kW
    min_max['elec_lights'] = [0,190]   #6 #kW
    min_max['Occ'] = [0,250]
    scale_delta, scale_hvac = 600 , 200


min_max['u_heat_stp'] = [15,25] 
min_max['u_cool_stp'] = [20,30] 
min_max['delta_T'] = [0,1] 
min_max['Stp_n'] = [0,1] 
min_max['mode'] = [0,1] 


def weights_init_(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight, gain=1)
        torch.nn.init.constant_(m.bias, 0)

class GaussianPolicy(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_dim, action_space=None):
        super(GaussianPolicy, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs, hidden_dim[0])
        self.linear2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        print (hidden_dim[2])
        print (num_actions)
        self.linear3 = nn.Linear(hidden_dim[1], hidden_dim[2])

        self.mean_linear = nn.Linear(hidden_dim[2], num_actions)
        self.log_std_linear = nn.Linear(hidden_dim[2], num_actions)

        self.apply(weights_init_)

        # action rescaling
        if action_space is None:
            self.action_scale = torch.tensor(1.)
            self.action_bias = torch.tensor(0.)
        else:
            self.action_scale = torch.FloatTensor(
                (action_space.high - action_space.low) / 2.)
            self.action_bias = torch.FloatTensor(
                (action_space.high + action_space.low) / 2.)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        mean = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, min=LOG_SIG_MIN, max=LOG_SIG_MAX)
        return mean, log_std

    def sample(self, state):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        normal = Normal(mean, std)
        x_t = normal.rsample()  # for reparameterization trick (mean + std * N(0,1))
        y_t = torch.tanh(x_t)
        action = y_t * self.action_scale + self.action_bias
        log_prob = normal.log_prob(x_t)
        # Enforcing Action Bound
        log_prob -= torch.log(self.action_scale * (1 - y_t.pow(2)) + epsilon)
        log_prob = log_prob.sum(1, keepdim=True)
        mean = torch.tanh(mean) * self.action_scale + self.action_bias
        return action, log_prob, mean

    def to(self, device):
        self.action_scale = self.action_scale.to(device)
        self.action_bias = self.action_bias.to(device)
        return super(GaussianPolicy, self).to(device)


class DeterministicPolicy(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_dim, action_space=None):
        super(DeterministicPolicy, self).__init__()
        self.linear1 = nn.Linear(num_inputs, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)

        self.mean = nn.Linear(hidden_dim, num_actions)
        self.noise = torch.Tensor(num_actions)

        self.apply(weights_init_)

        # action rescaling
        if action_space is None:
            self.action_scale = 1.
            self.action_bias = 0.
        else:
            self.action_scale = torch.FloatTensor(
                (action_space.high - action_space.low) / 2.)
            self.action_bias = torch.FloatTensor(
                (action_space.high + action_space.low) / 2.)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        mean = torch.tanh(self.mean(x)) * self.action_scale + self.action_bias
        return mean

    def sample(self, state):
        mean = self.forward(state)
        noise = self.noise.normal_(0., std=0.1)
        noise = noise.clamp(-0.25, 0.25)
        action = mean + noise
        return action, torch.tensor(0.), mean

    def to(self, device):
        self.action_scale = self.action_scale.to(device)
        self.action_bias = self.action_bias.to(device)
        self.noise = self.noise.to(device)
        return super(DeterministicPolicy, self).to(device)
    
class QNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_dim):
        super(QNetwork, self).__init__()

        # Q1 architecture
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_dim[0])
        self.linear2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.linear3 = nn.Linear(hidden_dim[1], hidden_dim[2])
        self.linear4 = nn.Linear(hidden_dim[2], 1)

        # Q2 architecture
        self.linear5 = nn.Linear(num_inputs + num_actions, hidden_dim[0])
        self.linear6 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.linear7 = nn.Linear(hidden_dim[1], hidden_dim[2])
        self.linear8 = nn.Linear(hidden_dim[2], 1)

        self.apply(weights_init_)

    def forward(self, state, action):
        xu = torch.cat([state, action], 1)
        
        x1 = F.relu(self.linear1(xu))
        x1 = F.relu(self.linear2(x1))
        x1 = F.relu(self.linear3(x1))
        x1 = self.linear4(x1)

        x2 = F.relu(self.linear5(xu))
        x2 = F.relu(self.linear6(x2))
        x2 = F.relu(self.linear7(x2))
        x2 = self.linear8(x2)

        return x1, x2
    

def e_load_checkpoint(ckpt_path, evaluate=False):       
    checkpoint = torch.load(ckpt_path)    
    e_policy.load_state_dict(checkpoint['policy_state_dict'])
    e_critic.load_state_dict(checkpoint['critic_state_dict'])
    e_critic_target.load_state_dict(checkpoint['critic_target_state_dict'])
    e_critic_optim.load_state_dict(checkpoint['critic_optimizer_state_dict'])
    e_policy_optim.load_state_dict(checkpoint['policy_optimizer_state_dict'])


In [26]:
e_obs_shape =  10 
e_act_shape = 2 

e_policy = GaussianPolicy(e_obs_shape, e_act_shape, args['hidden_size'], None).to(device)
e_critic_target = QNetwork(e_obs_shape, e_act_shape, args['hidden_size']).to(device)
e_critic = QNetwork(e_obs_shape, e_act_shape, args['hidden_size']).to(device=device)
e_critic_optim = Adam(e_critic.parameters(), lr=args['lr'])
e_policy_optim = Adam(e_policy.parameters(), lr=args['lr'])

# qf1_next_target, qf2_next_target = e_critic_target(next_state_batch, next_state_action)
# qf1, qf2 = e_critic(state_batch, action_batch) 

# qf1_loss = F.mse_loss(qf1, next_q_value) 
# qf2_loss = F.mse_loss(qf2, next_q_value)
# qf_loss = qf1_loss + qf2_loss

# e_critic_optim.zero_grad()
# qf_loss.backward()
# e_critic_optim.step()

# e_load_checkpoint(ckpt_path=, evaluate=False)


def init_rtp():
    rng = np.random.default_rng()
    price_rand = rng.uniform(size=(1, 24), low=0.16, high=0.25)[0]*0.8
    # print ("init_price:{}".format(price_rand))
    # print ()
    rtp = [0] * 24
    rtp[0] = price_rand[0]* np.random.uniform(0.5,0.7)  # 0-1
    rtp[1] = price_rand[1]* np.random.uniform(0.6,0.8)     # 1-2
    rtp[2] = price_rand[2]* np.random.uniform(0.6,0.8)      # 2-3
    rtp[3] = price_rand[3]* np.random.uniform(0.7,0.9)     # 3-4
    rtp[4] = price_rand[4]* np.random.uniform(0.7,0.9)       # 4-5
    rtp[5] = price_rand[5]* np.random.uniform(0.7,0.9)     # 5-6
    rtp[6] = price_rand[6]* np.random.uniform(0.7,0.9)       # 6-7
    rtp[7] = price_rand[7]* np.random.uniform(0.8,1.0)       # 7-8
    rtp[8] = price_rand[8]* np.random.uniform(0.8,1.0)       # 8-9
    rtp[9] = price_rand[9]* np.random.uniform(0.9,1.1)       # 9-10
    rtp[10]= price_rand[10]* np.random.uniform(0.9,1.2)       # 10-11
    rtp[11]= price_rand[11]* np.random.uniform(1.0,1.4)       # 11-12           
    rtp[12]= price_rand[12]* np.random.uniform(1.5,1.8)       # 12-13
    rtp[13]= price_rand[13]* np.random.uniform(1.8,2.2)       # 13-14
    rtp[14]= price_rand[14]* np.random.uniform(1.8,2.5)      # 14-15
    rtp[15]= price_rand[15]* np.random.uniform(1.5,2.2)       # 15-16
    rtp[16]= price_rand[16]* np.random.uniform(1.5,1.9)       # 16-17
    rtp[17]= price_rand[17]* np.random.uniform(1.0,1.5)       # 17-18
    rtp[18]= price_rand[18]* np.random.uniform(1.0,1.3)       #18-19
    rtp[19]= price_rand[19]* np.random.uniform(0.9,1.3)      # 19-20
    rtp[20]= price_rand[20]* np.random.uniform(0.9,1.2)       # 20-21
    rtp[21]= price_rand[21]* np.random.uniform(0.7,1.0)       # 21-22
    rtp[22]= price_rand[22]* np.random.uniform(0.5,0.7)       # 22-23
    rtp[23]= price_rand[23]* np.random.uniform(0.5,0.7)        #23-24
    # print (rtp)
    return rtp 

def get_rtp_price(rtp,hour):
    index = int(math.floor(hour))
    return rtp[index]

def round_down_to_half(n):
    return int(2 * n) / 2


def get_sol(sen_hou):
     # Parameters for the bell-shaped distribution
     peak_time = np.random.uniform(11,13)  # Peak at noon (12:00 PM)
     peak_value = np.random.uniform(200,700)/0.4
     std_deviation = 1.0  # Standard deviation
     # Calculate the corresponding values using a normal distribution
     bell_value = peak_value * norm.pdf(sen_hou, loc=peak_time, scale=std_deviation)

     return bell_value

def init_capacity_ev():
    no_of_occ = 3300
    percentage = 0.05

    ev_owners = int(no_of_occ*percentage)

    ev_data = pd.DataFrame(columns=['owner_no','capacity','start_time','end_time','start_soc','end_demand'])
    numbers = [20, 30, 40, 70, 100,120]
    probabilities = [0.25, 0.35, 0.25, 0.10,0.03,0.02]

    start_list =      [   6, 6.5, 7.0,7.5, 8.0, 8.5,    9,  9.5,   10,   11,   12, 13,   14]
    prob_start_list = [0.03,0.04,0.04,0.2, 0.4, 0.1, 0.05, 0.04, 0.035, 0.025, 0.02,0.01,0.01] 

    end_list =      [  13, 13.5, 14, 14.5,   15, 15.5,   16, 16.5,   17, 17.5, 18, 18.5, 19.0, 19.5, 20, 20.5,  21, 21.5, 22, 23, 24]
    prob_end_list = [ 0.5,  1.5,  1,    2,  3, 5,  6,  6, 20, 20,  12,   12,  4,  2.5,   1.25,   1, 1,  0.5, 0.5, 0.25,0]
    prob_end_list = [x/100 for x in prob_end_list]

    
    for owner in range(ev_owners):
        cap = np.random.choice(numbers,p=probabilities)
        new_row = {'owner_no':owner, 'capacity':cap,
                'start_time':np.random.choice(start_list,p=prob_start_list),
                'end_time':np.random.choice(end_list,p=prob_end_list),
                'start_soc': np.random.uniform(low=0.05,high=0.35)*cap,
                'end_demand': np.random.uniform(low=0.95,high=0.100)*cap
                }     
        # print (new_row)
        ev_data = pd.concat([ev_data, pd.DataFrame([new_row])], ignore_index=True)

    ev_data['discharge'] = ev_data['capacity']/5  
    ev_data['charge'] = ev_data['capacity']/3  
    times = np.arange(0, 24.5, 0.5).tolist()
    capacity_per_time = {time: ev_data.loc[(ev_data['start_time'] < time) & (ev_data['end_time'] >= time), 'capacity'].sum() for time in times}
    return capacity_per_time


1000
2


In [96]:
e_state_n

[0.20757852464477722,
 0.18777232096480959,
 0.5875667941038116,
 0.7680248812883541,
 0.7868818037957932,
 0.5986735814677058,
 0.946055141479575,
 1.3141238925383127,
 0.9686257885056884,
 0.5495520199154447]

In [97]:
action_n[:-1]


[0.0, 0.4535]

In [99]:
e_state_n

[0.20757852464477722,
 0.18777232096480959,
 0.5875667941038116,
 0.7680248812883541,
 0.7868818037957932,
 0.5986735814677058,
 0.946055141479575,
 1.3141238925383127,
 0.9686257885056884,
 0.5495520199154447]

In [98]:

qf1_target, qf2_target = e_critic_target(torch.tensor(e_state_n), torch.tensor(action_n[:-1]))
qf1, qf2 = e_critic([e_state_n], [action_n[:-1]])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [104]:
rtp = init_rtp()
rtp

[0.08751500103558234,
 0.10653704256122012,
 0.11524449977515756,
 0.15130308411854804,
 0.10680556224359292,
 0.14714899907507067,
 0.16368932152832555,
 0.15583386704391358,
 0.1351305394119396,
 0.18845739336450776,
 0.15930946918151057,
 0.1717825940362129,
 0.32255877455480175,
 0.31702914490225975,
 0.2896163462830814,
 0.2765827269219346,
 0.23576420889802613,
 0.16128213248139606,
 0.19792982003877577,
 0.14010780007953794,
 0.190937969620207,
 0.16121864707697425,
 0.11300001190286886,
 0.10199401399463126]

In [27]:
# sample generation 
n_samples = 50000
cnt =0 
actions_n = []
states_n = [] 
e_states_n = [] 
e_critic_q = [] 
e_critic_target_q = []

check = []

for iteration in range(n_samples):     

     print (iteration)

     if iteration%100==0:
          clear_output(wait=True)
          rtp = init_rtp()
          capacity_per_time =  init_capacity_ev()

     Temp_mean_p0 = np.random.uniform(17,28)
     Temp_mean_m1 = Temp_mean_p0 + np.random.uniform(-3,3)
     if np.random.uniform(0,1)<0.6:
          sen_hou = np.random.uniform(0,24)
     else:
          sen_hou = np.random.uniform(11,13)
     if sen_hou<6 and sen_hou>18:
          temp_OA_0 = np.random.uniform(16,26)
     else:
          temp_OA_0 = np.random.uniform(20,35)
     temp_OA_1 =  temp_OA_0 + np.random.uniform(-2,2)
     rh_OA = np.random.uniform(40,95)
     hir_sol_0 = get_sol(sen_hou)
     hir_sol_1 = get_sol((sen_hou+1)%24)
     rtp_0 = get_rtp_price(rtp,sen_hou) # rtp
     rtp_1 = get_rtp_price(rtp,(sen_hou+1)%24) # rtp
     rtp_2 = get_rtp_price(rtp,(sen_hou+2)%24) # rtp 
     rtp_3 = get_rtp_price(rtp,(sen_hou+3)%24) # rtp 
     ev_0 = capacity_per_time[round_down_to_half((sen_hou+0)%24)]
     ev_1 = capacity_per_time[round_down_to_half((sen_hou+0.5)%24)]
     ev_2 = capacity_per_time[round_down_to_half((sen_hou+1.0)%24)]
     ev_3 = capacity_per_time[round_down_to_half((sen_hou+1.5)%24)]
     ev_4 = capacity_per_time[round_down_to_half((sen_hou+2.0)%24)]
     soc = np.random.uniform(0,1)
     

     htp = 15
     clg = cool_stp(Temp_mean_p0,Temp_mean_m1,sen_hou,rh_OA,rtp_2)
     batt = batt_state_rbc(sen_hou,rtp_0,rtp_1,rtp_2)

     action = [htp,clg,batt] 

     state = [Temp_mean_p0,Temp_mean_m1,sen_hou,temp_OA_0,temp_OA_1,rh_OA,hir_sol_0,hir_sol_1,
          rtp_0,rtp_1,rtp_2,rtp_3,
          ev_0,ev_1,ev_2,ev_3,ev_4,soc]

     keys  = ['Temp_mean_p0','Temp_mean_m1','sen_hou','temp_OA','temp_OA','rh_OA','hir_sol','hir_sol',
          'rtp','rtp','rtp','rtp',
          'batt_cap','batt_cap','batt_cap','batt_cap','batt_cap','soc']
     act_keys = ['u_heat_stp','u_cool_stp','soc']
     e_state_keys = ['Temp_mean_p0','Temp_mean_m1','sen_hou','temp_OA','temp_OA','rh_OA','rtp','rtp','rtp','rtp']

     earlier_state = [Temp_mean_p0,Temp_mean_m1,sen_hou,temp_OA_0,temp_OA_1,rh_OA,rtp_0,rtp_1,rtp_2,rtp_3]

          
     state_n,action_n,e_state_n = [],[],[]
     for index,val in enumerate(state):
          # print (index)
          val_n  =  (val - min_max[keys[index]][0])/(min_max[keys[index]][1]-min_max[keys[index]][0])
          state_n.append(val_n)

     for index,val in enumerate(action):
          # print (index)
          act_val_n  =  (val - min_max[act_keys[index]][0])/(min_max[act_keys[index]][1]-min_max[act_keys[index]][0])
          action_n.append(act_val_n)

     for index,val in enumerate(earlier_state):
          # print (index)
          e_state_n_val_n  =  (val - min_max[e_state_keys[index]][0])/(min_max[e_state_keys[index]][1]-min_max[e_state_keys[index]][0])
          e_state_n.append(e_state_n_val_n)

     if rtp_0 > 0.25:
          cnt +=1
          check.append([state,action])

     # print (torch.tensor([e_state_n]))
     # print (torch.tensor([action_n[:-1]]))

     # qf1_target, qf2_target = e_critic_target(torch.tensor(e_state_n), torch.tensor(action_n[:-1]))
     # qf1, qf2 = e_critic([e_state_n], [action_n[:-1]]) 
     
     actions_n.append(action_n)
     states_n.append(state_n)
     # e_states_n.append(e_state_n)
     # e_critic_target_q.append([qf1_target,qf2_target])
     # e_critic_q.append([qf1,qf2])

np.save('actions_n.npy', np.array(actions_n)); np.save('states_n.npy', np.array(states_n))
# np.save('e_states_n.npy', np.array(e_states_n))
cnt

C:\Users\didso\AppData\Local\Temp\ipykernel_18016\771987519.py:102: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ev_data = pd.concat([ev_data, pd.DataFrame([new_row])], ignore_index=True)


KeyboardInterrupt: 

In [19]:
def gen_samples(no_samples):
    # Create a list of indices to sample from
    indices = list(range(len(states)))

    # Randomly shuffle the indices
    random.shuffle(indices)

    # Sample 40 elements with the same index from both arrays
    sampled_states = [states[i] for i in indices[:no_samples]]
    sampled_actions = [actions[i] for i in indices[:no_samples]]
    return sampled_states,sampled_actions

In [8]:
class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, init_w=3e-3, log_std_min=-20, log_std_max=2):
        super(PolicyNetwork, self).__init__()
        hidden_size = args['hidden_size']
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
                     
        self.linear1 = nn.Linear(num_inputs, hidden_size[0])
        self.linear2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.linear3 = nn.Linear(hidden_size[1], hidden_size[2])
        
        self.mean_linear = nn.Linear(hidden_size[2], num_actions)

        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        self.log_std_linear = nn.Linear(hidden_size[2], num_actions)

        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        
        mean    = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        
        return mean, log_std

In [20]:
import numpy as np ; import random
import gym; from gym import spaces
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
device = "cpu"



In [21]:
building_obs = ['Temp_mean_p0','Temp_mean_m1','sen_hou']
forecast_obs = {'temp_OA': [0,1],'rh_OA':[0],'hir_sol':{0,1}} 
price_struct = "rtp" 
price_obs = [0,1,2,3]
ev_avail_obs = [0,0.5,1.0,1.5,2.0]
state_dim_sac = len(building_obs) + sum(len(v) for v in forecast_obs.values()) + len(price_obs) + len(ev_avail_obs) + 1 

print (state_dim_sac)
sac_act_lower_bnds = [0,0,-1]; sac_act_upper_bnds = [1,1,1]

sac_obs_lower_bnds = np.zeros(state_dim_sac); sac_obs_upper_bnds = np.ones(state_dim_sac)

custom_observation_space = spaces.Box(low = sac_obs_lower_bnds,
                                       high = sac_obs_upper_bnds,
                                       dtype= np.float32) 
custom_action_space = spaces.Box(low  = np.array(sac_act_lower_bnds),
                                       high = np.array(sac_act_upper_bnds),
                                       dtype= np.float32) 


18


c:\Users\didso\.conda\envs\torch\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [11]:
args['hidden_size']

[1000, 1600, 1000]

In [110]:
states

array([[0.23959322, 0.10779197, 0.15540407, ..., 0.        , 0.        ,
        0.20415929],
       [0.55805489, 0.46549609, 0.40046877, ..., 1.03166667, 1.03833333,
        0.53014471],
       [0.46845386, 0.42076814, 0.0231451 , ..., 0.        , 0.        ,
        0.58233262],
       ...,
       [0.45740227, 0.40247397, 0.16756567, ..., 0.        , 0.        ,
        0.74428309],
       [0.54794202, 0.41060694, 0.86837179, ..., 0.01166667, 0.00833333,
        0.96812987],
       [0.41629326, 0.46580431, 0.07974822, ..., 0.        , 0.        ,
        0.19979441]])

In [12]:
policy = GaussianPolicy(custom_observation_space.shape[0], 3, args['hidden_size'], custom_action_space).to(device)
critic_target = QNetwork(custom_observation_space.shape[0], 3, args['hidden_size']).to(device)
critic = QNetwork(custom_observation_space.shape[0], 3, args['hidden_size']).to(device=device)
critic_optim = Adam(critic.parameters(), lr=args['lr'])
policy_optim = Adam(policy.parameters(), lr=args['lr'])

1000
3


In [13]:
def load_checkpoint(ckpt_path, evaluate=False):       
    checkpoint = torch.load(ckpt_path)    
    policy.load_state_dict(checkpoint['policy_state_dict'])
    critic.load_state_dict(checkpoint['critic_state_dict'])
    critic_target.load_state_dict(checkpoint['critic_target_state_dict'])
    critic_optim.load_state_dict(checkpoint['critic_optimizer_state_dict'])
    policy_optim.load_state_dict(checkpoint['policy_optimizer_state_dict'])

def save_checkpoint(ckpt_path=None):
    torch.save({'policy_state_dict': policy.state_dict(),
                'critic_state_dict': critic.state_dict(),
                'critic_target_state_dict': critic_target.state_dict(),
                'critic_optimizer_state_dict': critic_optim.state_dict(),
                'policy_optimizer_state_dict': policy_optim.state_dict()}, ckpt_path)

path_NN = '0_Data/Case_1d/02_NN/'
save_checkpoint(ckpt_path=path_NN+"checkpoints/sac_checkpoint_"+str(0)+"_")

In [22]:
policy_lr = 0.0001

epochs = 100


# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(policy.parameters(), lr=policy_lr)
ckpt_path = path_NN+"checkpoints/sac_checkpoint_"+str(0)+"_"
# checkpoint = torch.load(ckpt_path)

# Prepare the data
load_checkpoint(ckpt_path=path_NN+"checkpoints/sac_checkpoint_"+str(0)+"_")

"""alternate """
# X = torch.randn(n, 9)
# y = torch.randn(n, 2)
sample_gen_iter =50
# Training loop

for i in range(sample_gen_iter):
    sampled_states,sampled_actions = gen_samples(1000)
    X = torch.FloatTensor(sampled_states)
    y = torch.FloatTensor(sampled_actions)
    print ("gen new data")
    for epoch in range(epochs):
        # Forward pass
        action, _, _ = policy.sample(X)
        y_pred = action
        # y_pred = model(X)
        # print ("y_true: {};y_pred: {}".format(y,y_pred))

        # Compute and print loss
        loss = criterion(y_pred, y)
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')
        print (i)

        # Zero gradients, perform a backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        clear_output(wait=True)
        print ()

In [23]:
path_NN = '0_Data/Case_1d/02_NN/'
save_checkpoint(ckpt_path=path_NN+"checkpoints/sac_checkpoint_"+str(0)+"_")

In [58]:
X = torch.FloatTensor(states)
y = torch.FloatTensor(actions)
action, _, _ = policy.sample(X)

In [61]:
def get_rbc_case4c(sen_hou,rtp_0,rtp_1,rtp_2):
     htg = 15 
     if sen_hou>6 and sen_hou<19: 
          clg = 24
          if rtp_2>0.2:
              clg = 21 
          
          if rtp_0>0.25:
              clg = 24.5 
     else: 
          clg = 30 

     batt_act = batt_state_rbc(sen_hou,rtp_0,rtp_1,rtp_2)

     return [htg,clg,batt_act]


def batt_state_rbc(hours,rtp_0,rtp_1,rtp_2):
    if rtp_2>0.2 or rtp_1>0.2: 
        batt_act = 1.0       
    else:
        batt_act = 0           
    if hours>15:
        batt_act = 0.5 

    if rtp_0 >0.25:
        batt_act = -1  

    if hours>14:
        batt_act = 1 

    return batt_act 

[(7, 'g'),
 (5, 'e'),
 (10, 'j'),
 (1, 'a'),
 (3, 'c'),
 (4, 'd'),
 (6, 'f'),
 (9, 'i'),
 (2, 'b'),
 (8, 'h')]

In [67]:
array1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
array2 = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


    





array([0.41707928, 0.25847605, 0.44464433, ..., 1.02      , 1.02      ,
       0.29238616])

C:\Users\didso\AppData\Local\Temp\ipykernel_68060\504638642.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  X = torch.FloatTensor(sampled_states)


In [75]:
action, _, _ = policy.sample(X)

In [ ]:
action